# 1. Specify all parameters to be varied


In [5]:
CAD_variables = []
CAM_variables = [{
        'name' : 'layer height',
        'data type' : 'ratio',
        'test_values' : [x / 10.0 for x in range(5, 20, 5)], # i hate this so much, but we would need numpy arange to make it better
        'argname' : '--layer-height'
    },
    {
        'name' : 'fill pattern',
        'data type' : 'nominal',
        'categories' : ['triangles', 'concentric'],
        'test_values' : ['triangles', 'concentric'], # I would like them all to have the same named thing so I can call it
        'argname' : '--fill-pattern'
    }]
CAM_variables_laser = [{
        'name' : 'cut power',
        'data type' : 'ratio',
        'test_values' : [x for x in range(5, 100, 10)],
        'argtype' : 'profile'
    },
    {
        'name' : 'cut speed',
        'data type' : 'ratio',
        'test_values' : [x for x in range(5, 100, 10)],
        'argtype' : 'profile'
    }]
print_repetitions = 2

sample_post_process_variable = {
    'name' : 'paint type',
    'data type' : 'nominal',
    'categories' : ['lacquer', 'spraypaint'],
    'test_values' : ['lacquer', 'spraypaint'], # I would like them all to have the same named thing so I can call it
    'description' : 'use the {} on the object'
}
post_process_variables = [sample_post_process_variable]
post_process_repetitions = 1

sample_interaction_variable = {
    'name' : 'user push force level',
    'data type' : 'ratio',
    'test_values' : [x for x in range(10)],
    'instruction' : 'push on the button with force {} / 10' 
}
interaction_variables = [sample_interaction_variable]
interaction_repetitions = 1 # TODO: are interaction repetitions ever different from measurement repetitions?

measurement_variables = [{
        'name' : 'printed x-axis',
        'data type' : 'ratio'
    },
    {
        'name' : 'printed y-axis',
        'data type' : 'ratio'
    },
    {
        'name' : 'printed z-axis',
        'data type' : 'ratio'
    }
]
measurement_reptitions = 5

tea_hypothesis = (['material', 'measured z-axis' ], ['fill_pattern:concentric > triangle'])

# 2. CAD

In [ ]:
from fedt_print import make_labelled_stl
from fedt_laser import create_labelled # TODO: call them all generic things and just import the right ones

# FreeCAD create cube    
def geometry_function(App, side = '20 mm'):
    App.ActiveDocument.addObject("Part::Box","Box")
    App.ActiveDocument.ActiveObject.Label = "Cube"
    App.ActiveDocument.recompute()
    FreeCAD.ActiveDocument.getObject('Box').Width = side
    FreeCAD.ActiveDocument.getObject('Box').Length = side
    FreeCAD.ActiveDocument.getObject('Box').Height = side
    App.ActiveDocument.recompute()

# draw a circle in svg
def drawCircle(d):
    d.append(draw.Circle(-40, -10, 30,
                fill='none', stroke_width=1, stroke='red'))

# 3. CAM

In [ ]:
from fedt_print import slice_mesh
from fedt_laser import do_cam # TODO: rename them

# 4. Fabrication


In [ ]:
from fedt_print import execute_print
from fedt_laser import execute_lasercut # TODO: rename them

vars_to_labels = {}
i = 0
gcode_paths = []

# ???? how to make a nesting for structure whose depth is determined by the #
# of variables in the {post-process,CA{M,D}}_vars?
# ... obviously, use recursion instead of iteration :facepalm:
for h in variables['layer height']['values']:
    for p in variables['fill pattern']['categories']:
        for rep in range(print_repetitions):
            # make a labelled thing
            label = "L" + str(i)
            stl = make_labelled_stl(geometry_function, label_string=label)
            slicer_settings = {
                '--layer-height': h,
                '--fill-pattern': p
            }
            gcode_path = slice_mesh(stl, slicer_settings)
            i+=1
            gcode_paths.append(gcode_path)
            vars_to_labels[h+"_"+p+"//"+str(rep)] = label

# or send to simplyprint.io for later
for gcode_path in gcode_paths:
    execute_print(gcode_path)

# 5. Post-Processing

In [2]:
sample_post_process_variable = {
    'name' : 'paint type',
    'data type' : 'nominal',
    'categories' : ['lacquer', 'spraypaint'],
    'test_values' : ['lacquer', 'spraypaint'], # I would like them all to have the same named thing so I can call it
    'description' : 'use the {} on the object'
}

# TODO: I was thinking that I need to print out the things to do here, but I actually
# need to attach them somehow to the LABELS in the fabrication step. so that they know what to do to which.
# and then we need to print... something here... so the user knows what to do...

# 6. User Interaction

In [3]:
sample_interaction_variable = {
    'name' : 'user push force level',
    'data type' : 'ratio',
    'test_values' : [x for x in range(10)],
    'instruction' : 'push on the button with force {} / 10' 
}
interaction_variables = [sample_interaction_variable]
interaction_repetitions = 1

# TODO : figure out how to deal with this. is it true that we might need to generate the csv first? I... would think so.

# 7. Take measurements

In [4]:
from fedt_measure import create_experiment_csv
experiment_csv = create_experiment_csv(vars_to_labels, measurement_variables, measurement_repetitions)

print("now please fill " + experiment_csv + " with data")

NameError: name 'experiment_csv' is not defined

# 8. Tea-based hypothesis testing

In [ ]:
from fedt_tea import do_experiment

tea_results = do_experiment(measurement_variables,
                          tea_hypothesis,
                          experiment_csv,
                          CAD_variables,
                          CAM_variables,
                          post_process_variables,
                          interaction_variables)

# 9. Reporting

In [ ]:
from fedt_report import latex

latex(tea_results,
      tea_hypothesis,
      CAD_variables,
      CAM_variables,
      print_repetitions,
      post_process_variables,
      post_process_repetitions,
      interaction_variables,
      interaction_repetitions,
      measurement_variables,
      measurement_repetitions)